# Summary

In [1]:
from abstractor.train import get_training_batch as get_abstractor_training_batch
from abstractor.utils import AbstractorModel, AbstractorModelRNN
from abstractor.utils import obtain_initial_hidden_states
from bert.utils import obtain_sentence_embeddings
from bert.utils import obtain_word_embeddings
from data.utils import load_training_dictionaries
from extractor.train import get_training_batch as get_extractor_training_batch
from extractor.utils import ExtractorModel
from pytorch_transformers import BertModel
from pytorch_transformers import BertTokenizer
from rl_connection.utils import RLModel
from rl_connection.train import get_training_batch as get_rl_training_batch
from rouge import Rouge

import numpy as np
import torch

## Load data

In [2]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

In [3]:
data = load_training_dictionaries()

## Extractor

In [4]:
# Load extractor model:
extractor_model = ExtractorModel(bert_tokenizer, bert_model)
extractor_model_path = "results/models/extractor.pt"
extractor_model.load_state_dict(torch.load(extractor_model_path))

<All keys matched successfully>

In [5]:
documents, extraction_labels = get_extractor_training_batch(data, batch_size=2)

sentence_embeddings, mask = obtain_sentence_embeddings(
    extractor_model.bert_model, 
    extractor_model.bert_tokenizer, 
    documents
)

# Predict probability of extraction per sentence
extraction_probabilities = extractor_model(sentence_embeddings)

In [6]:
n_samples = len(documents)

for sample_idx in range(n_samples):
    n_to_extract = extraction_labels.sum(dim=1)[sample_idx].int() 
    ext_prob = extraction_probabilities[sample_idx] * mask[sample_idx]
    ext_sent_indicies = torch.topk(ext_prob, k=n_to_extract)[1]
    
    targets = np.array(documents[sample_idx])[extraction_labels[sample_idx][:len(documents[sample_idx])].numpy().astype(bool)]
    print("----> TARGET <----")
    for target in targets:
        print(f"{target}\n")
    print()
          
    print("----> PREDICTION <----")
    for x in np.array(documents[sample_idx])[ext_sent_indicies]:
        print(f"{x} \n")
    print("\n\n-------\n\n")

----> TARGET <----
newtown police say cayman was last seen wearing a gray down winter jacket , black ski pants and hiking boots . he could be in the radnor - wayne area , roughly 20 miles from philadelphia , or may have purchased a train ticket to philadelphia , according to an alert posted on facebook .

a message to families from the head of the shipley school , which cayman attends , read in part : " cayman 's sister savannah is in ninth grade at shipley and his parents , farid and becky , are terrific people . they have contacted police and are aware that we are sending you this email . we hope that cayman is ok and are saying our prayers . "


----> PREDICTION <----
a message to families from the head of the shipley school , which cayman attends , read in part : " cayman 's sister savannah is in ninth grade at shipley and his parents , farid and becky , are terrific people . they have contacted police and are aware that we are sending you this email . we hope that cayman is ok and

## Abstractor

In [7]:
# Load data:
abstractor_model = AbstractorModelRNN(bert_tokenizer, bert_model)
abstractor_model_path = "results/models/abstractor.pt"
abstractor_model.load_state_dict(torch.load(abstractor_model_path))

<All keys matched successfully>

In [8]:
source_documents, target_summaries = get_abstractor_training_batch(data, 1)

# Obtain embeddings
source_document_embeddings, source_mask, source_tokens = obtain_word_embeddings(
    abstractor_model.bert_model, abstractor_model.bert_tokenizer, source_documents, static_embeddings=False
)
target_summary_embeddings, target_mask, target_tokens = obtain_word_embeddings(
    abstractor_model.bert_model, abstractor_model.bert_tokenizer, target_summaries, static_embeddings=True
)

print(source_documents)
print()
print(target_summaries)

[['( cnn ) former vice president walter mondale was released from the mayo clinic on saturday after being admitted with influenza , hospital spokeswoman kelley luckstein said .', 'the 42nd vice president served under carter between 1977 and 1981 , and later ran for president , but lost to ronald reagan . but not before he made history by naming a woman , u.s. rep. geraldine a. ferraro of new york , as his running mate .']]

[['walter mondale was released from the mayo clinic on saturday , hospital spokeswoman said .', 'the former vice president , 87 , was treated for cold and flu symptoms .']]


In [9]:
# Obtain extraction probability for each word in vocabulary
extraction_probabilities, teacher_forcing = abstractor_model(
    source_document_embeddings,
    target_summary_embeddings,
    teacher_forcing_pct=0
)  # (batch_size, n_target_words, vocab_size)

vals, predicted_idx = torch.topk((extraction_probabilities), k=1, dim=2)

for x in [abstractor_model.bert_tokenizer.convert_ids_to_tokens(p) for p in predicted_idx.squeeze().tolist()]:
    print(f"{x}")
    print("")
    
print("------------")


# Obtain extraction probability for each word in vocabulary
extraction_probabilities, teacher_forcing = abstractor_model(
    source_document_embeddings,
    target_summary_embeddings,
    teacher_forcing_pct=1
)  # (batch_size, n_target_words, vocab_size)

vals, predicted_idx = torch.topk((extraction_probabilities), k=1, dim=2)

for x in [abstractor_model.bert_tokenizer.convert_ids_to_tokens(p) for p in predicted_idx.squeeze().tolist()]:
    print(f"{x}")
    print("")

walter

walter

walter

mon

##dale

was

released

from

the

mayo

clinic

on

saturday

spoke

##sw

said

said

said

said

said

said

said

said

said

said

said

said

said

said

said

said

said

said

said

said

------------
walter

walter

##dale

was

released

from

the

mayo

clinic

on

saturday

spoke

hospital

spoke

##sw

said

said

said

the

former

president

president

,

,

was

was

,

,

cold

and

flu

symptoms

.

[SEP]

.



## Reinforcement Learning

In [10]:
rl_model = RLModel(extractor_model, abstractor_model)
rl_model.load_state_dict(torch.load("results/models/rl.pt"))

<All keys matched successfully>

In [36]:
source_documents, target_summaries = get_rl_training_batch(data, batch_size=1)

# Obtain embeddings
source_sentence_embeddings, source_mask = obtain_sentence_embeddings(
    rl_model.extractor_model.bert_model,
    rl_model.extractor_model.bert_tokenizer,
    source_documents
)
stop_action_index = source_sentence_embeddings.shape[1]
target_summary_embeddings, target_mask, target_tokens = obtain_word_embeddings(
    rl_model.abstractor_model.bert_model,
    rl_model.abstractor_model.bert_tokenizer,
    target_summaries,
    static_embeddings=True
)

In [37]:
# Run trajectory
actions, log_probs, entropys, values, n_ext_sents = rl_model.sample_actions(
    source_sentence_embeddings,
    source_mask
)

# Obtain abstracted sentence from abstractor
predicted_tokens, word_probabilities = rl_model.create_abstracted_sentences(
    actions,
    source_documents,
    n_ext_sents=n_ext_sents,
    teacher_forcing_pct=0.0,
    target_summary_embeddings=target_summary_embeddings
)
        

RuntimeError: Expected tensor to have size 4 at dimension 0, but got size 1 for argument #2 'batch2' (while checking arguments for bmm)

In [33]:
# Look at extractions
for art_idx, doc_sentences in enumerate(actions):
    for sent_idx in doc_sentences[:-1]:
        print(source_documents[art_idx][sent_idx])
        print()
    print("\n\n-------\n\n")

before continuing his round with a dropped ball , the four - time major winner launched the 3 - iron used to play the offending shot into the water as well .

but when rory mcilroy pulled his second shot on the eighth hole of the wgc cadillac championship into a lake friday , he might as well have been channeling the much loved adam sandler character .

but when rory mcilroy pulled his second shot on the eighth hole of the wgc cadillac championship into a lake friday , he might as well have been channeling the much loved adam sandler character .



-------


mondale , 87 , was diagnosed after he went to the hospital for a routine checkup following a fever , former president jimmy carter said friday .

( cnn ) former vice president walter mondale was released from the mayo clinic on saturday after being admitted with influenza , hospital spokeswoman kelley luckstein said .

the 42nd vice president served under carter between 1977 and 1981 , and later ran for president , but lost to rona

In [34]:
# Look at abstractions
for predicted_abstraction in predicted_tokens:
    solution = list()
    for token in predicted_abstraction:
        solution.append(rl_model.abstractor_model.bert_tokenizer.ids_to_tokens[int(token)])
    print(" ".join(solution))
    print("\n\n")

##ib into ##ib of ##eum mc ##il at ##gc cadillac the reportedly the reportedly the reportedly the reportedly the reportedly go he ang northern irish ##man frustrated after pulling water hazard ##m grade grade grade



man suburban boston boston boston boston was , , , , , , , , , , , , , , , , , , , , , , , , , , , ,





In [35]:
target_summaries

[['rory mcilroy throws club into water at wgc cadillac championship .',
  'northern irishman frustrated after pulling shot into water hazard .'],
 ['walter mondale was released from the mayo clinic on saturday , hospital spokeswoman said .',
  'the former vice president , 87 , was treated for cold and flu symptoms .']]